In [1]:
# Import libs && packages.
import tensorflow as tf
import numpy as np
import pandas as pd
import scipy.optimize as opt
import statsmodels.api as sm
import matplotlib
import matplotlib.pyplot as plt

In [2]:
# Handle data.
df = pd.read_csv('../data/20190606180000-20190606190000.csv')
df=df.replace([np.inf, -np.inf], np.nan)
df=df.dropna();
df=df.round(decimals=5)  # round to one decimal after precision of devices
df['diff'] = (df['B_VIMIN'] - df['B:VIMIN'])

# Taking the data we're focused on.
df2=pd.DataFrame({'time': df['time_B:VIMIN'],'diff' : df['diff'], 'err' : df['B:IMINER']})
df2=df2.set_index(pd.DatetimeIndex(df2['time'])) # set index to time (not in-place operation)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (77,79,81,83,85,87) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# Construct model
def multilayer_perceptron(x, weights, biases):
    # Hidden layer with RELU activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    # # Hidden layer with RELU activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)

    # Output layer with linear activation
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

# Learning parameters
learning_rate = 0.0001
training_epochs = 10000
batch_size = 50
display_step = 1000

# Network Parameters
n_input =  1   # layer 0 (input layer) number of features
n_hidden_1 = 2 # layer 1 number of features
n_hidden_2 = 2 # layer 2 number of features



# Store layers weight && bias
weights = {
    'h1': tf.Variable(tf.random_uniform([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_uniform([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_uniform([n_hidden_2, 1]))
}
biases = {
    'b1': tf.Variable(tf.random_uniform([n_hidden_1])),
    'b2': tf.Variable(tf.random_uniform([n_hidden_2])),
    'out': tf.Variable(tf.random_uniform([1]))
}

x_data = tf.placeholder(tf.float32, [None, 1])
y_data = tf.placeholder(tf.float32, [None, 1])

# Construct model
pred = multilayer_perceptron(x_data, weights, biases)

# Build optimizer. Use mean-squared error for loss.
loss = tf.reduce_mean(tf.square(pred - y_data))
optimizer = tf.train.GradientDescentOptimizer(learning_rate)
train = optimizer.minimize(loss)

# Initialize variables. 
init = tf.global_variables_initializer()

In [4]:
def normalize_data(x):
  return (x - np.mean(x)) / np.std(x)

In [5]:
train_data = df2.sample(n = int(0.8*len(df2['time'])), random_state = 42)
test_data =  df2.drop(train_data.index)

In [6]:
train_data['diff'].apply(normalize_data)
train_data['err'].apply(normalize_data)
test_data['err'].apply(normalize_data)
test_data['err'].apply(normalize_data)
test_data.head()

,time,diff,err
time,,,
2019-06-06 23:00:00.170000076,2019-06-06 23:00:00.170000076,-0.00702,-0.07031
2019-06-06 23:00:00.636000156,2019-06-06 23:00:00.636000156,0.00917,0.09969
2019-06-06 23:00:00.702999830,2019-06-06 23:00:00.702999830,-0.01752,-0.16031
2019-06-06 23:00:00.835999966,2019-06-06 23:00:00.835999966,-0.02200,-0.21531
2019-06-06 23:00:01.102999926,2019-06-06 23:00:01.102999926,-0.00359,-0.03531


In [7]:
test_step=0
# Begin session.
sess = tf.Session()
sess.run(init)
print("epoch |   x   |  y    |      prediction     |     mean-squared error")
for step in range(training_epochs):
    train_samp = train_data.sample(n=batch_size, random_state = 42)
    x_in = np.array(train_samp['diff'].values)
    y_in = np.array(train_samp['err'].values)
    x_in = x_in.reshape(-1,1)
    y_in = y_in.reshape(-1,1)
    sess.run(train, feed_dict = {x_data: x_in, y_data: y_in})  
    if(step % display_step == 0):
        test_step += 1
        test_samp =  test_data.sample(n=1,random_state = step)
        x_test = np.array(test_samp['diff'].values)
        y_test = np.array(test_samp['err'].values)
        x_test = x_test.reshape(-1,1)
        y_test = y_test.reshape(-1,1)

        test_pred = sess.run(pred, feed_dict={x_data: x_test})
        test_loss = sess.run(loss, feed_dict={x_data: x_test, y_data: y_test})
        
        print("{0}     | {1}  | {2} |  {3} | {4}".format(test_step, round(x_test[0][0],4), round(y_test[0][0],4),np.round(test_pred[0][0],4), round(test_loss,4)))

epoch |   x   |  y    |      prediction     |     mean-squared error
1     | 0.031  | 0.3197 |  1.031000018119812 | 0.5059000253677368
2     | -0.014  | -0.1953 |  0.5102999806404114 | 0.49779999256134033
3     | -0.0371  | 0.1797 |  0.29510000348091125 | 0.013299999758601189
4     | -0.0266  | -0.2553 |  0.1868000030517578 | 0.19539999961853027
5     | -0.0068  | -0.0603 |  0.12559999525547028 | 0.03460000082850456
6     | 0.0074  | 0.0797 |  0.0885000005364418 | 9.999999747378752e-05
7     | -0.0163  | -0.1603 |  0.06120000034570694 | 0.04910000041127205
8     | -0.0011  | -0.0903 |  0.04830000177025795 | 0.019200000911951065
9     | 0.0063  | 0.0697 |  0.039799999445676804 | 0.0008999999845400453
10     | 0.0755  | 0.7697 |  0.04190000146627426 | 0.529699981212616


In [8]:
x_test_f = np.array(test_data['diff'].values)
y_test_f = np.array(test_data['err'].values)
x_test_f = x_test_f.reshape(-1,1)
y_test_f = y_test_f.reshape(-1,1)
test_samp = sess.run(pred, feed_dict={x_data: x_test})
test_loss = sess.run(loss, feed_dict={x_data: x_test, y_data: y_test})
print('mse over test set: ', test_loss)

mse over test set:  0.53520685


In [ ]:
tf.